In [7]:
import sys
sys.path.append("..")

## Load landmarks labelling

In [8]:
import pandas as pd

df = pd.read_pickle('../output/label.pkl')
df

coord                                                                x  \
file                                               landmark              
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1       -47.868675   
                                                   P10       10.347754   
                                                   P11       27.356030   
                                                   P12      -40.068095   
                                                   P2        -6.773722   
                                                   P3       -76.020657   
                                                   P4         7.909824   
                                                   P5       -66.313779   
                                                   P6       -15.675663   
                                                   P7       -11.845389   
                                                   P8        33.788057   
                                                   P9       -24.513464   

coord                                                                 y  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1       -102.082951   
                                                   P10      -141.225175   
                                                   P11       -66.791808   
                                                   P12       -98.773372   
                                                   P2       -102.914672   
                                                   P3       -115.615363   
                                                   P4       -105.273731   
                                                   P5       -133.365319   
                                                   P6        -75.853272   
                                                   P7        -63.623706   
                                                   P8       -143.022407   
                                                   P9       -149.428966   

coord                                                                 z  
file                                               landmark              
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1        -27.146627  
                                                   P10       191.482570  
                                                   P11       141.348713  
                                                   P12       148.841208  
                                                   P2        -29.897389  
                                                   P3         27.144024  
                                                   P4          7.795904  
                                                   P5         50.973984  
                                                   P6         80.303229  
                                                   P7        106.223576  
                                                   P8        163.758241  
                                                   P9        172.440759

In [9]:
from measure import label

file_ls = label.get_file_ls(df)
landmark_ls = label.get_landmark_ls(df)

## Local frame

In [10]:
import pyvista as pv
from measure import frame
from mesh4d.analyse import crave

file = file_ls[0]
mesh = crave.fix_pvmesh_disconnect(pv.read(file))

In [ ]:
import numpy as np

def axis_flip_to_align_link(axis, start, end):
    link = label.coord(df, file, end) - label.coord(df, file, start)
    cos = (link @ axis) / np.linalg.norm(link) / np.linalg.norm(axis)
    return np.sign(cos) * axis, np.sign(cos)

axis_flip_to_align_link(x_axis, 'P10', 'P1')

(array([ -58.21642895,   39.14222446, -218.62919764]), -1.0)

In [ ]:
import numpy as np

axes_frame, origin = frame.estimate_foot_frame(mesh, file, df)
frame.draw_axes(origin, axes_frame, mesh)

Widget(value="<iframe src='http://localhost:65124/index.html?ui=P_0x7f8749dcd250_1&reconnect=auto' style='widt…

In [ ]:
mesh_clip = frame.foot_clip(mesh, df, file)
mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)
mesh_local.plot()

NameError: name 'axes_frame' is not defined

In [ ]:
df_local = frame.df2local(df, axes_frame, origin)
df_local

coord                                                                 x  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1       -104.877373   
                                                   P10       124.412992   
                                                   P11        64.506712   
                                                   P12        63.916424   
                                                   P2        -98.482753   
                                                   P3        -56.299284   
                                                   P4        -58.881029   
                                                   P5        -27.729230   
                                                   P6         -1.043945   
                                                   P7         21.907593   
                                                   P8        103.399672   
                                                   P9        100.491941   

coord                                                                y  \
file                                               landmark              
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1         9.131839   
                                                   P10       -5.143694   
                                                   P11      -38.117774   
                                                   P12       32.423743   
                                                   P2       -31.516304   
                                                   P3        47.691094   
                                                   P4       -38.964888   
                                                   P5        44.186342   
                                                   P6        -5.919685   
                                                   P7        -6.284384   
                                                   P8       -32.843289   
                                                   P9        26.429013   

coord                                                                 z  
file                                               landmark              
/Users/knpob/Territory/Kolmo/data/DynaFootLite/... P1        341.001073  
                                                   P10       332.372554  
                                                   P11       272.467797  
                                                   P12       297.972022  
                                                   P2        344.796237  
                                                   P3        340.119885  
                                                   P4        339.308723  
                                                   P5        352.478265  
                                                   P6        292.771976  
                                                   P7        275.174926  
                                                   P8        341.806462  
                                                   P9        342.702867

## Metrics

In [ ]:
results = [
    {
        'file': 'description',
        'FL': 'foot length (mm)',
        'MBL': 'medial ball length (mm)',
        'LBL': 'lateral ball length (mm)',
        'ABW': 'anatomical ball width (mm)',
        'OBW': 'orthogonal ball width (mm)',
        'OHW': 'orthogonal heel width (mm)',
        'BH': 'ball heigh (mm)',
        'IH': 'instep height (mm)',
        'BA': 'ball angle (°)',
        'T1A': 'toe 1 angle (°)',
        'T5A': 'toe 5 angle (°)',
        'ABG': 'anatomical ball girth (mm)',
        'IG': 'instep girth (mm)',
    }
]

In [ ]:
from measure import metric

results.append(
    {
        'file': file,
        'FL': metric.fl(df_local, file),
        'MBL': metric.mbl(df_local, file),
        'LBL': metric.lbl(df_local, file),
        'ABW': metric.abw(df_local, file),
        'OBW': metric.obw(df_local, file),
        'OHW': metric.ohw(df_local, file),
        'BH': metric.bh(df_local, file),
        'IH': metric.ih(df_local, file),
        'BA': metric.ba(df_local, file),
        'T1A': metric.t1a(df_local, file),
        'T5A': metric.t5a(df_local, file),
        'ABG': metric.abg(df_local, file, mesh_local),
        'IG': metric.ig(df_local, file, mesh_local),
    }
)

In [ ]:
df_metric = pd.DataFrame(results).set_index('file')
df_metric

,FL,MBL,LBL,ABW,OBW,OHW,BH,IH,BA,T1A,T5A,ABG,IG
file,,,,,,,,,,,,,
description,foot length (mm),medial ball length (mm),lateral ball length (mm),anatomical ball width (mm),orthogonal ball width (mm),orthogonal heel width (mm),ball heigh (mm),instep height (mm),ball angle (°),toe 1 angle (°),toe 5 angle (°),anatomical ball girth (mm),instep girth (mm)
/Users/knpob/Territory/Kolmo/data/DynaFootLite/Fast/Sub05_Fast_L1_102_143.000006.obj,229.290364,183.294021,152.142222,89.766355,86.655983,59.272302,292.771976,275.174926,67.423662,164.578139,152.431568,124.034465,142.553302


In [ ]:
def plot_landmarks(scene, df_local, mesh_local, landamark_ls, frame_id=0, font_size=15, **kwargs):
    points = np.array([landmarks.markers[marker].coord[:, frame_id] for marker in landamark_ls])
    scene.add_points(points, render_points_as_spheres=True, point_size=10, **kwargs)

    offset = np.array([5, 5, 5])
    scene.add_point_labels(points + offset, landamark_ls, always_visible=True, font_size=font_size)

    return points